# Stance Detection for the Fake News Challenge

## Identifying Textual Relationships with Deep Neural Nets

## Step1: Load the given dataset

1. Mount the google drive

2. Import Glove embeddings

3. Import the test and train datasets

In [0]:
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


### Mount the google drive to access required project files

In [0]:
from google.colab import drive

In [0]:
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


#### Path for Project files on google drive

**Note:** You need to change this path according where you have kept the files in google drive. 

In [0]:
project_path = "/content/drive/My Drive/Datasets/Fake News Challenge/"

### Loading the Glove Embeddings

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'glove.6B.zip', 'r') as z:
  z.extractall()

### Load the dataset

1. Using [read_csv()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) in pandas load the given train datasets files **`train_bodies.csv`** and **`train_stances.csv`**

2. Using [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) command in pandas merge the two datasets based on the Body ID. 

Note: Save the final merged dataset in a dataframe with name **`dataset`**.

In [0]:
import pandas as pd

train_bodies = pd.read_csv(project_path+'train_bodies.csv')
train_stances = pd.read_csv(project_path+'train_stances.csv')

In [0]:
dataset = pd.merge(train_bodies[['Body ID', 'articleBody']],train_stances[['Body ID', 'Headline', 'Stance']], left_on='Body ID', right_on='Body ID', how='inner')


<h2> Check1:</h2>
  
<h3> You should see the below output if you run `dataset.head()` command as given below </h3>

In [0]:
dataset.head()

,Body ID,articleBody,Headline,Stance
0,0,A small meteorite crashed into a wooded area i...,"Soldier shot, Parliament locked down after gun...",unrelated
1,0,A small meteorite crashed into a wooded area i...,Tourist dubbed ‘Spider Man’ after spider burro...,unrelated
2,0,A small meteorite crashed into a wooded area i...,Luke Somers 'killed in failed rescue attempt i...,unrelated
3,0,A small meteorite crashed into a wooded area i...,BREAKING: Soldier shot at War Memorial in Ottawa,unrelated
4,0,A small meteorite crashed into a wooded area i...,Giant 8ft 9in catfish weighing 19 stone caught...,unrelated


## Step2: Data Pre-processing and setting some hyper parameters needed for model


#### Run the code given below to set the required parameters.

1. `MAX_SENTS` = Maximum no.of sentences to consider in an article.

2. `MAX_SENT_LENGTH` = Maximum no.of words to consider in a sentence.

3. `MAX_NB_WORDS` = Maximum no.of words in the total vocabualry.

4. `MAX_SENTS_HEADING` = Maximum no.of sentences to consider in a heading of an article.

In [0]:
MAX_NB_WORDS = 20000
MAX_SENTS = 20
MAX_SENTS_HEADING = 1
MAX_SENT_LENGTH = 20
VALIDATION_SPLIT = 0.2

### Download the `Punkt` from nltk using the commands given below. This is for sentence tokenization.

For more info on how to use it, read [this](https://stackoverflow.com/questions/35275001/use-of-punktsentencetokenizer-in-nltk).



In [0]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

### Tokenizing the text and loading the pre-trained Glove word embeddings for each token

Keras provides [Tokenizer API](https://keras.io/preprocessing/text/) for preparing text. Read it before going any further.

#### Import the Tokenizer from keras preprocessing text

In [0]:
from keras.preprocessing.text import Tokenizer

#### Initialize the Tokenizer class with maximum vocabulary count as `MAX_NB_WORDS` initialized at the start of step2. 

In [0]:
t = Tokenizer(num_words=MAX_NB_WORDS,filters= '!"#$%&()*+,-./:;<=>?@[\]^_`{|}\n“~')

#### Now, using fit_on_texts() from Tokenizer class, lets encode the data 

Note: We need to fit articleBody and Headline also to cover all the words.

In [0]:
t.fit_on_texts(dataset['articleBody'])
t.fit_on_texts(dataset['Headline'])

#### fit_on_texts() gives the following attributes in the output as given [here](https://faroit.github.io/keras-docs/1.2.2/preprocessing/text/).

* **word_counts:** dictionary mapping words (str) to the number of times they appeared on during fit. Only set after fit_on_texts was called.

* **word_docs:** dictionary mapping words (str) to the number of documents/texts they appeared on during fit. Only set after fit_on_texts was called.

* **word_index:** dictionary mapping words (str) to their rank/index (int). Only set after fit_on_texts was called.

* **document_count:** int. Number of documents (texts/sequences) the tokenizer was trained on. Only set after fit_on_texts or fit_on_sequences was called.



In [0]:
vocab_size = len(t.word_index) + 1
print(vocab_size)

27062


### Now, tokenize the sentences using nltk sent_tokenize() and encode the senteces with the ids we got form the above `t.word_index`

Initialise 2 lists with names `texts` and `articles`.

```
texts = [] to store text of article as it is.

articles = [] split the above text into a list of sentences.
```

In [0]:
from nltk import tokenize

texts = [] ## body
articles=[] ## body splitted into senteces and stored as a list of lists

## split into sentences
for idx in range(dataset['articleBody'].shape[0]):
    text = dataset['articleBody'][idx]
    texts.append(text)
    sentences = tokenize.sent_tokenize(text)
    articles.append(sentences)

## Check 2:

first element of texts and articles should be as given below. 

In [0]:
texts[0]

'A small meteorite crashed into a wooded area in Nicaragua\'s capital of Managua overnight, the government said Sunday. Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city\'s airport, the Associated Press reports. \n\nGovernment spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth." House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports. \nMurillo said Nicaragua will ask international experts to help local scientists in understanding what happened.\n\nThe crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee. He said it is still not clear if the meteorite disintegrated or was buried.\n\nHumbe

In [0]:
articles[0]

["A small meteorite crashed into a wooded area in Nicaragua's capital of Managua overnight, the government said Sunday.",
 "Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city's airport, the Associated Press reports.",
 'Government spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth."',
 'House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports.',
 'Murillo said Nicaragua will ask international experts to help local scientists in understanding what happened.',
 'The crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee.',
 'He said it is still not clear if the meteorite disintegrated or was bu

#### Now iterate through each article and each sentence to encode the words into ids using t.word_index

Here, to get words from sentence you can use `text_to_word_sequence` from keras preprocessing text.

1. Import text_to_word_sequence

2. Initialize a variable of shape (no.of articles, MAX_SENTS, MAX_SENT_LENGTH) with name `data` with zeros first (you can use numpy [np.zeros](https://docs.scipy.org/doc/numpy/reference/generated/numpy.zeros.html) to initialize with all zeros)and then update it while iterating through the words and sentences in each article.

In [0]:
from keras.preprocessing.text import text_to_word_sequence
import numpy as np

## for article body
data = np.zeros((len(texts), MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')
## word index encoding
for i, sentences in enumerate(articles):
    for j, sent in enumerate(sentences):
        if j < MAX_SENTS:
            wordTokens = text_to_word_sequence(sent)
            k = 0
            for _, word in enumerate(wordTokens):
                try:
                    if k < MAX_SENT_LENGTH and t.word_index[word] < MAX_NB_WORDS:
                      data[i, j, k] = t.word_index[word]
                      k = k + 1
                except:
#                       print(word)
                        pass

### Check 3:

Accessing first element in data should give something like given below.

In [0]:
data[0, :, :]

array([[    3,   479,   425,  7159,    81,     3,  3721,   334,     5,
         3871,   352,     4,  1430,  2946,     1,    89,    12,   464,
            0,     0],
       [  758,    96,  1046,     3,  2676,  1750,     7,   190,     3,
         1219,  1056,  1967,   701,   160,     1,  3020,   449,     1,
          553,   235],
       [   89,  1067,  4095,  2344,    12,     3,  1092,  3297,    19,
            1,    89,     2,  1789,     1,   520,  2008,    15,     9,
            3,  2912],
       [  186,  3629,   973,   200,  2551,    44,  6727,  1719,  1253,
            5, 13097, 17549,     1,   775,    30,   743,  3975,    67,
           86,     0],
       [ 2344,    12,  1556,    38,  1094,   354,   774,     2,   361,
          262,  1766,     5,  4432,    69,   493,     0,     0,     0,
            0,     0],
       [    1,   701,   190,    19,     1,   425,    32,     3,  7359,
            4,  2153,  1253,     6,     3,  5232,     4,  1219,  1253,
           12,  3354],
       [  

In [0]:
data[0, :, :].reshape(1, data.shape[1], data.shape[2]).shape

(1, 20, 20)

### Repeat the same process for the `Headings` as well. Use variables with names `texts_heading` and `articles_heading` accordingly.

In [0]:
texts_heading = [] ## body
articles_heading = [] ## body splitted into senteces and stored as a list of lists

## split into sentences
for idx in range(dataset['Headline'].shape[0]):
    text = dataset['Headline'][idx]
    texts_heading.append(text)
    sentences = tokenize.sent_tokenize(text)
    articles_heading.append(sentences)

## for article headings
data_heading = np.zeros((len(texts), MAX_SENTS_HEADING, MAX_SENT_LENGTH), dtype='int32')
## word index encoding
for i, sentences in enumerate(articles_heading):
    for j, sent in enumerate(sentences):
        if j < MAX_SENTS_HEADING:
            wordTokens = text_to_word_sequence(sent)
            k = 0
            for _, word in enumerate(wordTokens):
                try:
                    if k < MAX_SENT_LENGTH and t.word_index[word] < MAX_NB_WORDS:
                        data_heading[i, j, k] = t.word_index[word]
                        k = k + 1
                except:
                    pass

### Now the features are ready, lets make the labels ready for the model to process.

### Convert labels into one-hot vectors

You can use [get_dummies](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) in pandas to create one-hot vectors.

In [0]:
## one-hot vectors for labels                  
labels = dataset['Stance']
targets = pd.Series(labels)
one_hot = pd.get_dummies(targets, sparse = True)
one_hot_labels = np.asarray(one_hot)
labels = one_hot_labels

### Check 4:

The shape of data and labels shoould match the given below numbers.

In [0]:
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (49972, 20, 20)
Shape of label tensor: (49972, 4)


### Shuffle the data

In [0]:
## get numbers upto no.of articles
indices = np.arange(data.shape[0])
## shuffle the numbers
np.random.shuffle(indices)

In [0]:
## shuffle the data
data = data[indices]
data_heading = data_heading[indices]
## shuffle the labels according to data
labels = labels[indices]

### Split into train and validation sets. Split the train set 80:20 ratio to get the train and validation sets

In [0]:
## no.of validation samples = 0.2 of original size
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

In [0]:
## for train and validation samples
x_train = data[:-nb_validation_samples]
x_heading_train = data_heading[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
x_heading_val = data_heading[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

### Check 5:

The shape of x_train, x_val, y_train and y_val should match the below numbers.

In [0]:
print(x_train.shape)
print(y_train.shape)

print(x_val.shape)
print(y_val.shape)

(39978, 20, 20)
(39978, 4)
(9994, 20, 20)
(9994, 4)


### Create embedding matrix with the glove embeddings


Run the below code to create embedding_matrix which has all the words and their glove embedding if present in glove word list.

In [0]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('./glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 100))


for word, i in t.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

Loaded 400000 word vectors.


## Try different sequential models 

### Import layers from Keras to build the model

In [0]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, TimeDistributed, Activation
from keras.layers import Flatten, Permute, merge, Input
from keras.layers import Embedding
from keras.models import Model
from keras.layers import Input, Dense, multiply, concatenate, Dropout

In [0]:
MAX_SENT_LENGTH = 20
MAX_SENTS = 20
hidden_size = 100


from keras.layers import Embedding, Input, Dense, LSTM, GRU, Bidirectional, TimeDistributed

sentence_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
print(sentence_input)
embedded_sequences = Embedding(output_dim=hidden_size, input_dim=vocab_size, input_length=(MAX_SENT_LENGTH,), weights=[embedding_matrix])(sentence_input)

l_lstm = Bidirectional(LSTM(100, return_sequences=True))(embedded_sequences)
l_dense = TimeDistributed(Dense(100))(l_lstm)
l_dense = Flatten()(l_dense)
sentEncoder = Model(sentence_input, l_dense)

body_input = Input(shape=(MAX_SENTS,MAX_SENT_LENGTH,), dtype='int32')
print(body_input)
body_encoder = TimeDistributed(sentEncoder)(body_input)
print(body_encoder)
l_lstm_sent = Bidirectional(LSTM(100, return_sequences=True))(body_encoder)
l_dense_sent = TimeDistributed(Dense(100))(l_lstm_sent)
l_dense_sent = Flatten()(l_dense_sent)


heading_input = Input(shape=(MAX_SENTS_HEADING, MAX_SENT_LENGTH,), dtype='int32')
heading_embedded_sequences = Embedding(output_dim=hidden_size, input_dim=vocab_size, \
                                       input_length=(MAX_SENTS_HEADING, MAX_SENT_LENGTH,), weights=[embedding_matrix])(heading_input)
h_dense = Dense(100, activation='relu')(heading_embedded_sequences)
h_flatten = Flatten()(h_dense)
article_output = concatenate([l_dense_sent, h_flatten], name='concatenate_heading')

news_vector = Dense(100, activation='relu')(article_output)
preds = Dense(4, activation='softmax')(news_vector)
model = Model([body_input, heading_input], [preds])

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

print("model fitting - Hierachical LSTM network")
model.fit([x_train, x_heading_train], [y_train], validation_data=([x_val, x_heading_val], [y_val]), epochs=10, batch_size=50)

model fitting - Hierachical LSTM network
Train on 39978 samples, validate on 9994 samples
Epoch 1/10
39978/39978 [==============================] - 224s 6ms/step - loss: 0.2460 - acc: 0.9077 - val_loss: 0.2798 - val_acc: 0.8995
Epoch 2/10
39978/39978 [==============================] - 218s 5ms/step - loss: 0.1789 - acc: 0.9341 - val_loss: 0.2698 - val_acc: 0.9126
Epoch 3/10
39978/39978 [==============================] - 220s 6ms/step - loss: 0.1369 - acc: 0.9498 - val_loss: 0.2541 - val_acc: 0.9133
Epoch 4/10
39978/39978 [==============================] - 218s 5ms/step - loss: 0.1101 - acc: 0.9612 - val_loss: 0.2216 - val_acc: 0.9228
Epoch 5/10
39978/39978 [==============================] - 219s 5ms/step - loss: 0.0921 - acc: 0.9671 - val_loss: 0.2147 - val_acc: 0.9340
Epoch 6/10
39978/39978 [==============================] - 218s 5ms/step - loss: 0.0748 - acc: 0.9728 - val_loss: 0.2207 - val_acc: 0.9357
Epoch 7/10
39978/39978 [==============================] - 219s 5ms/step - loss: 0.

### Adding Attention layer

In [0]:
# def Attention(inputs):
#     input_dim = int(inputs.shape[2])
#     v = Permute((2, 1))(inputs)
#     v = Dense(20, activation='softmax')(v)
#     attention_probs = Permute((2, 1), name='attention_porbs')(v)
#     attention_output = multiply([inputs, attention_probs], name='mul_attention')
#     return attention_output

from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers

class AttLayer(Layer):
    def __init__(self, attention_dim):
        self.init = initializers.get('normal')
        self.supports_masking = True
        self.attention_dim = attention_dim
        super(AttLayer, self).__init__()

    def build(self, input_shape):
        assert len(input_shape) == 3
        self.W = K.variable(self.init((input_shape[-1], self.attention_dim)))
        self.b = K.variable(self.init((self.attention_dim, )))
        self.u = K.variable(self.init((self.attention_dim, 1)))
        self.trainable_weights = [self.W, self.b, self.u]
        super(AttLayer, self).build(input_shape)

    def compute_mask(self, inputs, mask=None):
        return mask

    def call(self, x, mask=None):
        # size of x :[batch_size, sel_len, attention_dim]
        # size of u :[batch_size, attention_dim]
        # uit = tanh(xW+b)
        uit = K.tanh(K.bias_add(K.dot(x, self.W), self.b))
        ait = K.dot(uit, self.u)
        ait = K.squeeze(ait, -1)

        ait = K.exp(ait)

        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            ait *= K.cast(mask, K.floatx())
        ait /= K.cast(K.sum(ait, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        ait = K.expand_dims(ait)
        weighted_input = x * ait
        output = K.sum(weighted_input, axis=1)

        return output

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

#### Building Model with Attention

### Hierarchical Attention Network

In [0]:
MAX_SENT_LENGTH = 20
MAX_SENTS = 20
hidden_size = 100


from keras.layers import Embedding, Input, Dense, LSTM, GRU, Bidirectional, TimeDistributed

sentence_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
embedded_sequences = Embedding(output_dim=hidden_size, input_dim=vocab_size, input_length=(MAX_SENT_LENGTH,), weights=[embedding_matrix])(sentence_input)

l_lstm = Bidirectional(LSTM(100, return_sequences=True))(embedded_sequences)
l_dense = TimeDistributed(Dense(100))(l_lstm)
l_att = AttLayer(100)(l_dense)
sentEncoder = Model(sentence_input, l_att)

body_input = Input(shape=(MAX_SENTS,MAX_SENT_LENGTH), dtype='int32')
body_encoder = TimeDistributed(sentEncoder)(body_input)
l_lstm_sent = Bidirectional(LSTM(100, return_sequences=True))(body_encoder)
l_dense_sent = TimeDistributed(Dense(100))(l_lstm_sent)
l_att_sent = AttLayer(100)(l_dense_sent)


heading_input = Input(shape=(MAX_SENTS_HEADING, MAX_SENT_LENGTH,), dtype='int32')
heading_embedded_sequences = Embedding(output_dim=hidden_size, input_dim=vocab_size, input_length=(MAX_SENTS_HEADING, MAX_SENT_LENGTH,), weights=[embedding_matrix])(heading_input)
h_dense = Dense(10, activation='relu')(heading_embedded_sequences)
h_flatten = Flatten()(h_dense)
article_output = concatenate([l_att_sent, h_flatten], name='concatenate_heading')

news_vector = Dense(100, activation='relu')(article_output)
preds = Dense(4, activation='softmax')(news_vector)
model = Model([body_input, heading_input], [preds])

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

print("model fitting - Hierachical attention network")
model.fit([x_train, x_heading_train], [y_train], validation_data=([x_val, x_heading_val], [y_val]), epochs=10, batch_size=50)

model fitting - Hierachical attention network
Train on 39978 samples, validate on 9994 samples
Epoch 1/10
39978/39978 [==============================] - 218s 5ms/step - loss: 0.6488 - acc: 0.7604 - val_loss: 0.4787 - val_acc: 0.8147
Epoch 2/10
39978/39978 [==============================] - 212s 5ms/step - loss: 0.3954 - acc: 0.8478 - val_loss: 0.4117 - val_acc: 0.8379
Epoch 3/10
39978/39978 [==============================] - 211s 5ms/step - loss: 0.2775 - acc: 0.8958 - val_loss: 0.2700 - val_acc: 0.8991
Epoch 4/10
39978/39978 [==============================] - 212s 5ms/step - loss: 0.2019 - acc: 0.9251 - val_loss: 0.2321 - val_acc: 0.9144
Epoch 5/10
39978/39978 [==============================] - 212s 5ms/step - loss: 0.1551 - acc: 0.9422 - val_loss: 0.2256 - val_acc: 0.9218
Epoch 6/10
39978/39978 [==============================] - 212s 5ms/step - loss: 0.1240 - acc: 0.9553 - val_loss: 0.1994 - val_acc: 0.9303
Epoch 7/10
39978/39978 [==============================] - 212s 5ms/step - los